### Debt Repayment Plan

#### Basic Case

- Data
    - **N<sub>i</sub>**  : Monthly income during period **i**
    - **E<sub>i</sub>**  : Monthly expense during period **i**
    - **M<sub>i</sub>**  : Monthly required savings during period **i**
    - **I**              : Interest paid on loan
    

- Decision Variables
    - **S<sub>i</sub>**  : Monthly savings during period **i**
    - **R<sub>i</sub>**  : Debt paid during month **i**
    - **X<sub>i</sub>**  : Debt owed at the beginning of month **i**

In [1]:
using JuMP
import HiGHS

In [2]:
# Data
data = Dict(
    "debt" => Dict(
        "loan" => 10000,
        "period" => 10,
        "interest" => 6.8 / 100
    ),
    "net_income" => 4500
)
periods = 1:data["debt"]["period"]


1:10

In [3]:
function set_model_variables(model::Model, periods::UnitRange, NET_INCOME::Int)
    # Monthly savings
    Savings = @variable(model, 0 <= S[periods] <= NET_INCOME);

    # Monthly debt reimbursed
    Payments = @variable(model, 0 <= P[periods] <= NET_INCOME);

    # Monthly balance
    Balance = @variable(model, B[cat(periods, periods[end] + 1, dims = 1)] >= 0);
    
    return Savings, Payments, Balance
end

set_model_variables (generic function with 1 method)

In [4]:
function set_model_constraints( 
        model::Model, 
        debt::Dict,
        NET_INCOME::Int,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balance::JuMP.Containers.DenseAxisArray
    )
    loan_label = "loan"; period_label = "period"; interest_label = "interest";
    if haskey(debt, loan_label ) && haskey( debt, period_label ) && haskey( debt, interest_label )
        # Debt owed at the beginning of period 1
        @constraint( model, Balance[1] == debt[loan_label] )

        # Ensure that all debt is paid off by the grace period
        @constraint( model, Balance[debt[period_label]] == 0 )

        # Debt owed the beginning of period 2...end
        @constraint( model, [i in periods], Balance[i + 1] == (1 + debt[interest_label]) * (Balance[i] - Payments[i]) )

        # Monthly expense can't exceed net income
        @constraint( model, [i in periods], Savings[i] + Payments[i] <= NET_INCOME )
    end
end

set_model_constraints (generic function with 1 method)

#### Solve Model

In [5]:
model = Model( HiGHS.Optimizer )

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [6]:
Savings, Payments, Balance = set_model_variables( model, periods, data["net_income"] )

(1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{VariableRef}:
 S[1]
 S[2]
 S[3]
 S[4]
 S[5]
 S[6]
 S[7]
 S[8]
 S[9]
 S[10], 1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{VariableRef}:
 P[1]
 P[2]
 P[3]
 P[4]
 P[5]
 P[6]
 P[7]
 P[8]
 P[9]
 P[10], 1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
And data, a 11-element Vector{VariableRef}:
 B[1]
 B[2]
 B[3]
 B[4]
 B[5]
 B[6]
 B[7]
 B[8]
 B[9]
 B[10]
 B[11])

In [7]:
set_model_constraints( model, data["debt"], data["net_income"], Savings, Payments, Balance )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 S[1] + P[1] <= 4500.0
 S[2] + P[2] <= 4500.0
 S[3] + P[3] <= 4500.0
 S[4] + P[4] <= 4500.0
 S[5] + P[5] <= 4500.0
 S[6] + P[6] <= 4500.0
 S[7] + P[7] <= 4500.0
 S[8] + P[8] <= 4500.0
 S[9] + P[9] <= 4500.0
 S[10] + P[10] <= 4500.0

In [8]:
@objective(model, Max, sum(Savings))

S[1] + S[2] + S[3] + S[4] + S[5] + S[6] + S[7] + S[8] + S[9] + S[10]

In [9]:
print( model )

In [10]:
function solve_infeasible(model)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        return objective_value(model)
    else
        @warn("The model was not solved correctly.")
        return nothing
    end
end

solve_infeasible(model)

Presolving model
16 rows, 24 cols, 39 nonzeros
10 rows, 18 cols, 27 nonzeros
10 rows, 18 cols, 27 nonzeros
Presolve : Reductions: rows 10(-12); columns 18(-13); elements 27(-25)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
         11    -3.4532568000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 11
Objective value     :  3.4532568000e+04
HiGHS run time      :          0.00


34532.568

In [14]:
using Printf
function print_payment_plan( 
        model::Model,
        debt::Dict,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balance::JuMP.Containers.DenseAxisArray
    )
    @printf("| %-10s | %-15s | %-10s | %-10s | %-15s | %-10s | %-12s |\n",
        "Period", "Initial Balance", "Payment", "Savings", 
        "Int. Rate (%)", "Interest", "End Balance"
    )
    for i in periods
        @printf("| %10d | %15.2f | %10.2f | %10.2f | %15.2f | %10.2f | %12.2f |\n", 
            i,
            value(Balance[i]), 
            value(Payments[i]), 
            value(Savings[i]),
            debt["interest"],
            if i == 1 0 else value(Payments[i]) - (value(Balance[i - 1] ) - value(R[i - 1])) end,
            value(X[i]) - value(R[i])
        )
    end
end;

In [12]:
if termination_status(model) == OPTIMAL
    print_payment_plan(model, data["debt"], S, R, X)
end

LoadError: UndefVarError: S not defined

In [13]:
@printf("%119s", '\U2111')

                                                                                                                      ℑ